# Keras Data Loaders and Augmentator

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Image-Data-Generator" data-toc-modified-id="Image-Data-Generator-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Image Data Generator</a></span><ul class="toc-item"><li><span><a href="#SAME-DIRECTORY-from-TEST-and-TRAIN" data-toc-modified-id="SAME-DIRECTORY-from-TEST-and-TRAIN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>SAME DIRECTORY from TEST and TRAIN</a></span></li></ul></li></ul></div>

**OPCIONES**
    * ImageDataGenerator + imgaug
    * ImageDataGenerator + tf.keras.layers.experimental.preprocessing

Librerias

* **ImageDataGenerator:**
   * CPU
   * No easy Custom preprocessing techniques
   * From directory
   * https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator?version=nightly
   * https://keras.io/api/preprocessing/image/#imagedatasetfromdirectory-function
   * https://towardsdatascience.com/keras-data-generators-and-how-to-use-them-b69129ed779c
   * https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
   * https://stackoverflow.com/questions/42443936/keras-split-train-test-set-when-using-imagedatagenerator
   * https://www.pyimagesearch.com/2019/07/08/keras-imagedatagenerator-and-data-augmentation/
   * **TESTEAR**: gen.fit(X_train)
X_train_aug = gen.flow(X_train, y_train_cat, seed=0)
 
* **imgaug**
     * Specifically, the ***data augmentation generator method*** takes the batches of images generated by a ImageDataGenerator for training or validation and adds to the input images the selected augmentations, preserving an original copy of them to
     * GaussianBlur method, CutOut, CoarseDropout
* **tf.keras.layers.experimental.preprocessing**
    * GPU
    * Lo de Santiago de Twitter
    * https://keras.io/guides/preprocessing_layers/
    * https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing
    
* **tf.keras.preprocessing.image.load_img**
    * Hace qcargar imagen
    
    
    https://www.kaggle.com/cdeotte/dog-autoencoder

## Image Data Generator
### SAME DIRECTORY from TEST and TRAIN
**A0**
```python
train_datagen = keras.preprocessing.image.ImageDataGenerator(..., validation_split=0.2) # OJO set validation split

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(PATH,
                                                 subset='training',
                                                 class_mode='input', #OJOOOO,
                                                 color_mode='grayscale',
                                                 shuffle=True,
                                                 #save_to_dir='dir prueba' para testear el augmentado,
                                                 #save_format='png',
                                                 target_size = (64, 64),
                                                 batch_size = 32)

validation_set = validation_datagen.flow_from_directory(PATH,
                                                        subset='validation',
                                                        class_mode='input', #OJOOOO
                                                        target_size = (64, 64),
                                                        batch_size = 32)
model = create_model()

# steps_per_epoch should be (number of training images total // batch_size) 
# validation_steps should be (number of validation images total // batch_size) 
model.fit(training_set,
          steps_per_epoch = 8000,
          epochs = 5,
          validation_data = validation_set,
          validation_steps = 2000)
```

**Alternativa 1**:
```python
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_data_generator(data_dir,
                       data_augment=False,
                       batch_size=BATCH_SIZE,
                       target_size=(100, 100),
                       color_mode='rgb',
                       class_mode='input', #OJOOOO
                       shuffle=True):
    if data_augment:
        datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   validation_split=0.2,#this is the trick
                                   horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    generator = datagen.flow_from_directory(data_dir,
                                          target_size=target_size,
                                          color_mode=color_mode,
                                          batch_size=batch_size,
                                          shuffle=shuffle,
                                          class_mode=class_mode)
    return generator

train_generator = image_data_generator('Your_DataBase_Path',data_augment=True)
```
**Alternativa2**
```python

train_datagen = ImageDataGenerator(..., validation_split=0.2) # OJO set validation split

train_generator = train_datagen.flow_from_directory(
    batch_size=batch_size,
    train_data_dir='full_dataset',
    target_size=(img_height, img_width),
    subset='training') # OJO set as training data

validation_generator = train_datagen.flow_from_directory(
    batch_size=batch_size,
    train_data_dir='full_dataset', # OJO same directory as training data
    target_size=(img_height, img_width),
    subset='validation') # OJO set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)
```

In [ ]:
training_generator = ImageDataGenerator()
validation_generator = ImageDataGenerator()
training_images = training_generator.flow_from_directory(training_directory, class_mode='input')
validation_images = validation_generator.flow_from_directory(validation_directory, class_mode='input')